<a href="https://colab.research.google.com/github/karlmanalo/DS-Unit-2-Applied-Modeling/blob/master/module3-permutation-boosting/NFL_Combine_Again.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Importing Athos, Porthos and Aramis. Quick rabbit hole but I thought the third
# musketeer was D'Artagnan because I was misremembering Slumdog Millionaire

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [57]:
# Installing 0.25.0 version of Pandas to make sure Pandas Profiling runs

!pip install pandas==0.25

In [0]:
# Defining df

df = pd.read_csv('https://raw.githubusercontent.com/karlmanalo/Lambda-School-Unit-1-Project/master/combine_data_since_2000_PROCESSED_2018-04-26.csv')

In [59]:
# Python & Pandas versions for reproducibility's sake

import sys
print('Python version:', sys.version)
print('Pandas version:', pd.__version__)

Python version: 3.6.9 (default, Nov  7 2019, 10:44:02) 
[GCC 8.3.0]
Pandas version: 0.25.0


In [0]:
# Creating target 'Drafted'

df['Drafted'] = [0 if np.isnan(x) else 1 for x in df['Pick']]

In [0]:
# Featuring BMI column

df['BMI'] = 703 * df['Wt'] / df['Ht']**2

In [62]:
# Running Pandas Profiling

import pandas_profiling
pandas_profiling.ProfileReport(df)

/usr/local/lib/python3.6/dist-packages/pandas_profiling/describe.py:392: FutureWarning:

The join_axes-keyword is deprecated. Use .reindex or .reindex_like on the result to achieve the same functionality.



Number of variables,18
Number of observations,6218
Total Missing (%),14.1%
Total size in memory,874.5 KiB
Average record size in memory,144.0 B
Numeric,11
Categorical,3
Boolean,1
Date,0
Text (Unique),1
Rejected,2


In [0]:
# Defining majority class, defining y_pred as array of majority class

majority_class = df['Drafted'].mode()[0]

y_pred = [majority_class] * len(df['Drafted'])

In [64]:
# Importing and calculating accuracy score of majority class

from sklearn.metrics import accuracy_score

accuracy_maj_class = accuracy_score(df['Drafted'], y_pred)

print(f'The baseline accuracy of guessing the majority class for every prediction would be {accuracy_maj_class}.')

The baseline accuracy of guessing the majority class for every prediction would be 0.6011579285944033.


In [0]:
# Defining function to wrangle data set.

def wrangle(X):
  
  # Creating copy of X to prevent SettingWithCopy warnings
  X = X.copy()

  # Dropping unusable_variance
  unusable_variance = ['Player', 'Pfr_ID']
  X = X.drop(columns=unusable_variance)

  # Dropping columns with data leakage
  data_leakage = ['AV', 'Team', 'Round', 'Pick']
  X = X.drop(columns=data_leakage)

  return X

In [66]:
# Wrangling df

df_wrangled = wrangle(df)
df_wrangled.head()

,Pos,Ht,Wt,Forty,Vertical,BenchReps,BroadJump,Cone,Shuttle,Year,Drafted,BMI
0,OLB,76,252,4.55,NaN,NaN,NaN,NaN,NaN,2000,1,30.671053
1,RB,72,218,4.58,NaN,NaN,NaN,NaN,NaN,2000,1,29.562886
2,OT,76,334,5.56,25.0,23.0,94.0,8.48,4.98,2000,1,40.651316
3,TE,74,253,4.97,29.0,NaN,104.0,7.29,4.49,2000,0,32.479730
4,CB,74,206,4.55,34.0,NaN,123.0,7.18,4.15,2000,1,26.445946


In [67]:
# Running Pandas Profiling for wrangled data set

pandas_profiling.ProfileReport(df_wrangled)

/usr/local/lib/python3.6/dist-packages/pandas_profiling/describe.py:392: FutureWarning:

The join_axes-keyword is deprecated. Use .reindex or .reindex_like on the result to achieve the same functionality.



Number of variables,12
Number of observations,6218
Total Missing (%),12.7%
Total size in memory,583.1 KiB
Average record size in memory,96.0 B
Numeric,9
Categorical,1
Boolean,1
Date,0
Text (Unique),0
Rejected,1


In [0]:
import plotly.express as px

In [0]:
%matplotlib inline

In [70]:
# I know these are not great visuals :/

fig = px.scatter(df_wrangled, x='Forty', y='Drafted')
fig.show()

In [71]:
# I know these are not great visuals :/

fig = px.scatter(df_wrangled, x='Ht', y='Drafted')
fig.show()

In [0]:
train = df_wrangled[ (df_wrangled['Year'] >=2000) & (df_wrangled['Year'] < 2015)]
val = df_wrangled[ (df_wrangled['Year'] >=2015) & (df_wrangled['Year'] < 2017)]
test = df_wrangled[ df_wrangled['Year'] >=2017]

In [0]:
# def drop_date(X):
  
#   X = X.copy()

#   X = X.drop(columns='Year')

#   return X

In [0]:
# train = drop_date(train)
# val = drop_date(val)
# test = drop_date(test)

In [0]:
target = 'Drafted'

X_train = train.drop(columns=target)
y_train = train[target]

X_val = val.drop(columns=target)
y_val = val[target]

X_test = test

In [107]:
pip install category_encoders

In [0]:
import category_encoders as ce
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

In [109]:
pipeline = make_pipeline(
    ce.OrdinalEncoder(),
    SimpleImputer(strategy='median'),
    XGBClassifier(n_estimators=100, random_state=42, n_jobs=-1)
)

pipeline.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('ordinalencoder',
                 OrdinalEncoder(cols=['Pos'], drop_invariant=False,
                                handle_missing='value', handle_unknown='value',
                                mapping=[{'col': 'Pos', 'data_type': dtype('O'),
                                          'mapping': OLB     1
RB      2
OT      3
TE      4
CB      5
K       6
P       7
FS      8
OG      9
ILB    10
DE     11
SS     12
DT     13
QB     14
WR     15
C      16
FB     17
LS     18
NaN    -2
dtype: int64}],
                                return_df=True, verbose=0)),
                ('simpleimputer',
                 SimpleImputer(add_indicator=F...
                 XGBClassifier(base_score=0.5, booster='gbtree',
                               colsample_bylevel=1, colsample_bynode=1,
                               colsample_bytree=1, gamma=0, learning_rate=0.1,
                               max_delta_step=0, max_depth=3,
                               m

In [110]:
y_pred = pipeline.predict(X_val)
print('Validation Accuracy', accuracy_score(y_val, y_pred))

Validation Accuracy 0.6880733944954128


In [111]:
transformers = make_pipeline(
    ce.OrdinalEncoder(),
    SimpleImputer(strategy='median')
)

X_train_transformed = transformers.fit_transform(X_train)
X_val_transformed = transformers.transform(X_val)

model = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
model.fit(X_train_transformed, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=-1, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)

In [112]:
pip install eli5

In [113]:
import eli5
from eli5.sklearn import PermutationImportance

permuter = PermutationImportance(
    model,
    scoring='accuracy',
    n_iter=5,
    random_state=42
)

permuter.fit(X_val_transformed, y_val)

PermutationImportance(cv='prefit',
                      estimator=RandomForestClassifier(bootstrap=True,
                                                       ccp_alpha=0.0,
                                                       class_weight=None,
                                                       criterion='gini',
                                                       max_depth=None,
                                                       max_features='auto',
                                                       max_leaf_nodes=None,
                                                       max_samples=None,
                                                       min_impurity_decrease=0.0,
                                                       min_impurity_split=None,
                                                       min_samples_leaf=1,
                                                       min_samples_split=2,
                                                       min_weight_fr

In [114]:
feature_names = X_val.columns.tolist()
pd.Series(permuter.feature_importances_, feature_names).sort_values()

Year         0.000000
BroadJump    0.004893
BenchReps    0.005810
Ht           0.008257
Pos          0.008563
Vertical     0.017737
Shuttle      0.019878
Cone         0.025994
BMI          0.032722
Forty        0.066361
Wt           0.082263
dtype: float64

In [115]:
eli5.show_weights(
    permuter,
    top=None,
    feature_names=feature_names
)

Weight,Feature
0.0823 ± 0.0209,Wt
0.0664 ± 0.0163,Forty
0.0327 ± 0.0169,BMI
0.0260 ± 0.0043,Cone
0.0199 ± 0.0084,Shuttle
0.0177 ± 0.0154,Vertical
0.0086 ± 0.0178,Pos
0.0083 ± 0.0094,Ht
0.0058 ± 0.0174,BenchReps
0.0049 ± 0.0148,BroadJump
